## 날릴 거
- matchid, groupid
- kill
- rankpoint
- 차량 파괴
- maxplace
- 
## 기준
- MAE
- matchtype

## 고려사항
- 주행, 수영, 도보 이동 거리는 합할까..?

### 솔로, 듀오, 듀오 1인칭, 스쿼드 제외 나머지

In [1]:
# 모듈 로딩
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#데이터 준비

data = pd.read_csv('../datas/train_V2.csv')
sample = pd.read_csv('../datas/sample_submission_V2.csv')

sample

,Id,winPlacePerc
0,9329eb41e215eb,1
1,639bd0dcd7bda8,1
2,63d5c8ef8dfe91,1
3,cf5b81422591d1,1
4,ee6a295187ba21,1
...,...,...
1934169,a316c3a13887d5,1
1934170,5312146b27d875,1
1934171,fc8818b5b32ad3,1
1934172,a0f91e35f8458f,1


In [3]:
#컬럼명 확인하고 필요 없는 거 날리기

data.columns

Index(['Id', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt', 'DBNOs',
       'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc'],
      dtype='object')

In [4]:
usingdata = data.drop(['Id','groupId','matchId','killPoints','kills','maxPlace','numGroups','rankPoints','rideDistance','roadKills','swimDistance','vehicleDestroys','winPoints'], axis = 1)

In [5]:
usingdata.columns

Index(['assists', 'boosts', 'damageDealt', 'DBNOs', 'headshotKills', 'heals',
       'killPlace', 'killStreaks', 'longestKill', 'matchDuration', 'matchType',
       'revives', 'teamKills', 'walkDistance', 'weaponsAcquired',
       'winPlacePerc'],
      dtype='object')

In [6]:
# 쓸 데이터 : 매치타입 컬럼의 특수 타입(노멀 스쿼드 1인칭, 노멀 솔로 1인칭) 추출
# 크래시와 플레어는 놀이 개념이 큰 맵이라 고려해야 할 요소가 다르기 때문에 제외

usingdata['matchType'].unique() 

array(['squad-fpp', 'duo', 'solo-fpp', 'squad', 'duo-fpp', 'solo',
       'normal-squad-fpp', 'crashfpp', 'flaretpp', 'normal-solo-fpp',
       'flarefpp', 'normal-duo-fpp', 'normal-duo', 'normal-squad',
       'crashtpp', 'normal-solo'], dtype=object)

In [7]:
mask = usingdata['matchType'].isin(['normal-squad-fpp','normal-solo-fpp', 'normal-duo-fpp','normal-solo','normal-duo','normal-squad'])
others = usingdata[mask]
others = others.drop(['matchType'], axis = 1)

In [8]:
others = others.reset_index()
others

,index,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killStreaks,longestKill,matchDuration,revives,teamKills,walkDistance,weaponsAcquired,winPlacePerc
0,265,0,3,335.90,1,0,4,16,1,5.65,1950,0,0,3187.00,5,0.8000
1,1292,6,0,1750.00,0,6,2,7,3,69.88,1034,0,0,1137.00,43,0.7143
2,1351,2,0,655.20,0,2,0,11,3,1.48,492,0,0,29.57,9,0.4737
3,1417,2,0,245.20,0,0,10,45,1,19.80,697,0,0,16.72,15,0.4167
4,1649,2,6,89.14,0,0,7,32,0,0.00,1952,1,0,2563.00,12,0.9333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25381,4446353,1,4,1300.00,0,5,5,2,3,109.90,833,0,0,186.30,13,1.0000
25382,4446375,1,0,396.00,3,0,3,35,1,77.15,854,0,0,225.80,17,0.6000
25383,4446682,0,0,41.95,0,0,0,48,1,43.38,599,0,0,0.00,4,0.9434
25384,4446749,0,5,0.00,0,0,5,35,0,0.00,1921,0,0,2398.00,4,0.4167


In [9]:
# 피처와 타겟 분리

target = others[others.columns[-1]]
feature = others[others.columns[:-1]]

In [10]:
target

0        0.8000
1        0.7143
2        0.4737
3        0.4167
4        0.9333
          ...  
25381    1.0000
25382    0.6000
25383    0.9434
25384    0.4167
25385    1.0000
Name: winPlacePerc, Length: 25386, dtype: float64

In [11]:
#others.isnull().sum()
others.isna().sum()

index              0
assists            0
boosts             0
damageDealt        0
DBNOs              0
headshotKills      0
heals              0
killPlace          0
killStreaks        0
longestKill        0
matchDuration      0
revives            0
teamKills          0
walkDistance       0
weaponsAcquired    0
winPlacePerc       0
dtype: int64

In [12]:
feature.weaponsAcquired
#feature.walkDistance

0         5
1        43
2         9
3        15
4        12
         ..
25381    13
25382    17
25383     4
25384     4
25385     4
Name: weaponsAcquired, Length: 25386, dtype: int64

In [13]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(feature, target, test_size = 0.3)

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(xtrain)
scaler.fit(xtest)

xtrain_scaled = scaler.transform(xtrain)
xtest_scaled = scaler.transform(xtest)

In [15]:
from sklearn.linear_model import LinearRegression

In [16]:
lrmodel = LinearRegression()
lrmodel.fit(xtrain_scaled, ytrain)

LinearRegression()

In [17]:
lrmodel.score(xtrain_scaled, ytrain)

0.3564603647782606

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

In [19]:
rf = RandomForestRegressor(random_state=0)

In [20]:
rf.fit(xtrain_scaled, ytrain)

RandomForestRegressor(random_state=0)

In [21]:
rf.score(xtrain_scaled, ytrain)

0.9278000314502156

In [22]:
rf.score(xtest_scaled, ytest)

0.4920635866925318

In [23]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [24]:
y_pred = rf.predict(xtest_scaled)

In [25]:
mse = mean_squared_error(ytest, y_pred)
mae = mean_absolute_error(ytest, y_pred)
r2 = r2_score(ytest, y_pred)

In [26]:
print(f' [MeanSquaredError]: {mse}\n [Mean Absolute Error]: {mae}\n [R2] : {r2} ')

 [MeanSquaredError]: 0.052135679385011166
 [Mean Absolute Error]: 0.17402394078256303
 [R2] : 0.4920635866925318 
